In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [28]:
# Load the CSV file into a Pandas dataframe
df = pd.read_csv('Train_set.csv')

# Display dimensions of the dataset
print(df.shape)

(87554, 189)


In [29]:
# Display first 5 rows of the dataset
df.head()

,ID,T0,T1,T2,T3,T4,T5,T6,T7,T8,...,T178,T179,T180,T181,T182,T183,T184,T185,T186,Class
0,0,0.965812,0.792023,0.116809,0.000000,0.162393,0.213675,0.264957,0.247863,0.270655,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,1.000000,0.597015,0.000000,0.109453,0.094527,0.084577,0.074627,0.094527,0.114428,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,0.831382,0.714286,0.491803,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,1.000000,0.837705,0.236066,0.037705,0.252459,0.329508,0.319672,0.306557,0.304918,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,0.868590,0.448718,0.490385,0.477564,0.461538,0.455128,0.416667,0.304487,0.182692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [30]:
# Display all values in Class column
df['Class'].unique()

array([0, 4, 1, 2, 3])

In [31]:
# Count the number of missing values in each column
print(df.isnull().sum())

ID          0
T0        474
T1        919
T2       1327
T3       1711
         ... 
T183     4898
T184     4956
T185     4928
T186     4909
Class       0
Length: 189, dtype: int64


In [32]:
# Display the avg of missing values in each column and sort them
print(df.isnull().mean().sort_values(ascending=False))

T76      0.058558
T77      0.058387
T69      0.058318
T75      0.058261
T78      0.058044
           ...   
T2       0.015156
T1       0.010496
T0       0.005414
ID       0.000000
Class    0.000000
Length: 189, dtype: float64


In [33]:
# Display the avg of missing values in each row and sort them
print(df.isnull().mean(axis=1).sort_values(ascending=False))

41960    0.105820
46987    0.105820
18223    0.105820
23724    0.105820
34985    0.105820
           ...   
16218    0.005291
37495    0.005291
22590    0.005291
33657    0.005291
40458    0.005291
Length: 87554, dtype: float64


In [34]:
# Impute NaN values with the median value of each column
df.fillna(df.median(), inplace=True)

In [35]:
# Display dimensions of the dataset
df.head()

,ID,T0,T1,T2,T3,T4,T5,T6,T7,T8,...,T178,T179,T180,T181,T182,T183,T184,T185,T186,Class
0,0,0.965812,0.792023,0.116809,0.000000,0.162393,0.213675,0.264957,0.247863,0.270655,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,1.000000,0.597015,0.000000,0.109453,0.094527,0.084577,0.074627,0.094527,0.114428,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,0.831382,0.714286,0.491803,0.166197,0.148022,0.158824,0.145332,0.144465,0.149909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,1.000000,0.837705,0.236066,0.037705,0.252459,0.329508,0.319672,0.306557,0.304918,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,0.868590,0.448718,0.490385,0.477564,0.461538,0.455128,0.416667,0.304487,0.182692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [36]:
# Separate the ID and class columns from the input features
ids = df['ID']
y = df['Class']
X = df.drop(['ID', 'Class'], axis=1)

# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape the input features into a 3D array for input to the LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [38]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, (h_n, c_n) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [39]:
class LSTMModel_Drop(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.25):
        super(LSTMModel_Drop, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, (h_n, c_n) = self.lstm(self.dropout(x), (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


In [40]:
class LSTMModel_Deep(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.25):
        super(LSTMModel_Deep, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers=1, batch_first=True, dropout=dropout)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, (h_n, c_n) = self.lstm1(self.dropout(x), (h0, c0))
        out, (h_n, c_n) = self.lstm2(out, (h_n, c_n))
        out = self.fc(out[:, -1, :])
        return out


In [41]:
# Convert the preprocessed data to PyTorch tensors
X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y.values).long()

# Create a PyTorch DataLoader to handle batching and shuffling of the data
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize variables for early stopping
best_val_loss = float('inf')
patience = 5
counter = 0

# Define the device to use for training (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the model and move it to the device
model = LSTMModel_Deep(input_size=1, hidden_size=128, num_layers=1, output_size=5).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
num_epochs = 40
for epoch in range(num_epochs):
    for X_batch, y_batch in dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Zero out the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    # Print the loss and validation accuracy at the end of each epoch
    with torch.no_grad():
        total_loss = 0
        total_correct = 0
        total_samples = 0
        for X_val, y_val in dataloader:
            X_val = X_val.to(device)
            y_val = y_val.to(device)
            outputs = model(X_val)
            total_loss += criterion(outputs, y_val).item() * X_val.shape[0]
            total_correct += (outputs.argmax(dim=1) == y_val).sum().item()
            total_samples += X_val.shape[0]
        val_loss = total_loss / total_samples
        val_acc = total_correct / total_samples
    #print(f"Epoch {epoch+1}/{num_epochs}: train_loss={loss.item():.4f} val_loss={val_loss:.4f} val_acc={val_acc:.4f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping after epoch {epoch+1}.")
            break
    
    # Print the loss and validation accuracy at the end of each epoch
    print(f"Epoch {epoch+1}/{num_epochs}: train_loss={loss.item():.4f} val_loss={val_loss:.4f} val_acc={val_acc:.4f}")
    
    if counter >= patience:
        break

/home/gscognami/.local/lib/python3.8/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/40: train_loss=1.5985 val_loss=0.6356 val_acc=0.8274
Epoch 2/40: train_loss=1.4565 val_loss=0.6342 val_acc=0.8283
Epoch 3/40: train_loss=2.3220 val_loss=0.5109 val_acc=0.8416
Epoch 4/40: train_loss=1.9642 val_loss=0.4833 val_acc=0.8452
Epoch 5/40: train_loss=0.0479 val_loss=0.3651 val_acc=0.8907
Epoch 6/40: train_loss=0.0260 val_loss=0.2854 val_acc=0.9161
Epoch 7/40: train_loss=0.0297 val_loss=0.2261 val_acc=0.9333
Epoch 8/40: train_loss=0.0706 val_loss=0.1893 val_acc=0.9423
Epoch 9/40: train_loss=0.0159 val_loss=0.1701 val_acc=0.9508
Epoch 10/40: train_loss=0.0201 val_loss=0.1525 val_acc=0.9576
Epoch 11/40: train_loss=0.0264 val_loss=0.1470 val_acc=0.9594
Epoch 12/40: train_loss=0.0032 val_loss=0.1382 val_acc=0.9607
Epoch 13/40: train_loss=0.0166 val_loss=0.1210 val_acc=0.9666
Epoch 14/40: train_loss=0.0200 val_loss=0.1266 val_acc=0.9647
Epoch 15/40: train_loss=0.0049 val_loss=0.1217 val_acc=0.9657
Epoch 16/40: train_loss=0.0866 val_loss=0.1076 val_acc=0.9695
Epoch 17/40: trai

In [42]:
# Define the path where you want to save the model
model_path = 'my_lstm_model_deep.pth'

# Save the model
torch.save(model.state_dict(), model_path)

In [43]:
# Load the saved model
loaded_model = LSTMModel_Deep(1, 128, 1, 5)
loaded_model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [44]:
# Load test data into a pandas DataFrame
test_data = pd.read_csv("Test_set.csv")

In [45]:
# Impute NaN values with the median value of each column
test_data.fillna(test_data.median(), inplace=True)

In [46]:
test_data.head()

,ID,T0,T1,T2,T3,T4,T5,T6,T7,T8,...,T177,T178,T179,T180,T181,T182,T183,T184,T185,T186
0,0,0.966581,0.802057,0.077121,0.000000,0.161954,0.244216,0.254499,0.262211,0.264782,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.000000,0.644444,0.031111,0.026667,0.151111,0.128889,0.080000,0.080000,0.084444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1.000000,0.965318,0.734104,0.462428,0.271676,0.156069,0.150289,0.173410,0.173410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1.000000,0.998792,0.960145,0.896135,0.898551,0.873188,0.766908,0.591787,0.327295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.009836,0.000000,0.163934,0.242623,0.327869,0.350820,0.331148,0.331148,0.354098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# Drop irrelevant columns
test_data = test_data.drop(columns=["ID"])

In [48]:
# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(test_data)

# Reshape the input features into a 3D array for input to the LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Convert the numpy array to a PyTorch tensor
test_tensor = torch.Tensor(X)

# Create a DataLoader object for the test data
test_loader = torch.utils.data.DataLoader(test_tensor, batch_size=32, shuffle=False)


In [49]:
# Set the device to be used for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the same device as the input tensor
loaded_model = loaded_model.to(device)

In [50]:
# Set the model to evaluation mode
loaded_model.eval()

# Create an empty list to store the predicted classes
pred_classes = []

# Loop through the test data in batches using the test loader
for X_batch in test_loader:
    # Move the batch to the device
    X_batch = X_batch.to(device)

    # Compute the model's predicted class for the batch
    outputs = loaded_model(X_batch)
    _, predicted = torch.max(outputs.data, 1)

    # Convert the predicted class tensor to a numpy array
    predicted = predicted.cpu().numpy()

    # Append the predicted classes to the list
    pred_classes.append(predicted)

# Convert the list of predicted classes to a numpy array
pred_classes = np.concatenate(pred_classes)

# Create a new pandas DataFrame with the predicted classes and an ID column (if necessary)
ids = np.arange(len(pred_classes))
df = pd.DataFrame({'ID': ids, 'Pred_Class': pred_classes})

# Save the DataFrame as a CSV file
df.to_csv('sample_submission_deep.csv', index=False)



In [51]:
result = pd.read_csv("sample_submission_deep.csv")

result.head()

,ID,Pred_Class
0,0,0
1,1,0
2,2,0
3,3,2
4,4,0


In [52]:
import csv

# Open the first file
with open('sample_submission_deep.csv', 'r') as file1:
    reader1 = csv.reader(file1)
    rows1 = [row for row in reader1]

# Open the second file
with open('sample_submission_deep_best.csv', 'r') as file2:
    reader2 = csv.reader(file2)
    rows2 = [row for row in reader2]
 
c = 0
# Compare the rows in each file
for i in range(len(rows1)):
    if rows1[i] != rows2[i]:
        print(f"Rows {i+1} are different:")
        print(f"File 1: {rows1[i]}")
        print(f"File 2: {rows2[i]}")
        c+=1

print(f"Total number of different rows: {c}")


Rows 85 are different:
File 1: ['83', '2']
File 2: ['83', '0']
Rows 142 are different:
File 1: ['140', '2']
File 2: ['140', '4']
Rows 154 are different:
File 1: ['152', '2']
File 2: ['152', '0']
Rows 156 are different:
File 1: ['154', '3']
File 2: ['154', '4']
Rows 262 are different:
File 1: ['260', '2']
File 2: ['260', '0']
Rows 267 are different:
File 1: ['265', '0']
File 2: ['265', '1']
Rows 282 are different:
File 1: ['280', '1']
File 2: ['280', '0']
Rows 294 are different:
File 1: ['292', '1']
File 2: ['292', '0']
Rows 391 are different:
File 1: ['389', '3']
File 2: ['389', '2']
Rows 394 are different:
File 1: ['392', '0']
File 2: ['392', '4']
Rows 482 are different:
File 1: ['480', '0']
File 2: ['480', '4']
Rows 523 are different:
File 1: ['521', '4']
File 2: ['521', '0']
Rows 531 are different:
File 1: ['529', '1']
File 2: ['529', '0']
Rows 572 are different:
File 1: ['570', '0']
File 2: ['570', '4']
Rows 610 are different:
File 1: ['608', '2']
File 2: ['608', '0']
Rows 616 are 